# Events Extractions

In [1]:
## Requirements

In [2]:
!pip3 install tqdm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [3]:
!pip3 install daterangeparser

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [4]:
!pip3 install -U spacy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [5]:
!python3 -m spacy download en_core_web_sm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 17.5 MB/s eta 0:00:00m eta 0:00:010:00:01
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
import re

import spacy
import requests
import re
import IPython
from daterangeparser import parse
from tqdm import tqdm
import pandas as pd 
import json

In [7]:
nlp = spacy.load("en_core_web_sm")

In [8]:
def dep_subtree(token, dep):
    deps =[child.dep_ for child in token.children]
    child=next(filter(lambda c: c.dep_==dep, token.children), None)
    if child != None:
        return " ".join([c.text for c in child.subtree])
    else:
        return ""

In [9]:
# to remove citations, e.g. "[91]" as this makes problems with spaCy
p = re.compile(r'\[\d+\]')

def extract_events_spacy(line):
    '''Function based on code from https://www.qualicen.de/natural-language-processing-timeline-extraction-with-regexes-and-spacy/
    '''

    line=p.sub('', line)
    events = []
    doc = nlp(line)
    for ent in filter(lambda e: e.label_=='DATE',doc.ents):
        try:
            start,end = parse(ent.text)
        except:
          # could not parse the dates, hence ignore it
          continue
        current = ent.root
        while current.dep_ != "ROOT":
            current = current.head
        desc = " ".join(filter(None,[
                                     dep_subtree(current,"nsubj"),
                                     dep_subtree(current,"nsubjpass"),
                                     dep_subtree(current,"auxpass"),
                                     dep_subtree(current,"amod"),
                                     dep_subtree(current,"det"),
                                     current.text, 
                                     dep_subtree(current,"acl"),
                                     dep_subtree(current,"dobj"),
                                     dep_subtree(current,"attr"),
                                     dep_subtree(current,"advmod")]))
        events = events + [(start.isoformat(),ent.text,desc)]
        events.sort(key=lambda tup: tup[0])
    return events

In [10]:
extract_events_spacy("Jan 2001 test Dec 2010, this that May 2005")

[('2001-01-01T00:00:00', 'Jan 2001', 'test'),
 ('2005-05-01T00:00:00', 'May 2005', 'test'),
 ('2010-12-01T00:00:00', 'Dec 2010', 'test')]

In [11]:
input_json_file = 'data/kym.json'  # Tarmo edited
output_json_file = 'data/events_url.json'  # Tarmo edited

In [12]:
df = pd.read_json(input_json_file)

In [13]:
memes  = df[['title', 'url','content']][df['category']=='Meme']

In [14]:
c = 0
l = 0
out = []
for i in tqdm(range(len(memes))):
    title = memes.title.iloc[i]
    # url = memes.url.iloc[i]  # Tarmo added
    origin = ""
    spread = ""
    
    if len(list(memes['content'].iloc[i].keys()))>0:
        try:
            origin = " ".join(memes['content'].iloc[i]["origin"]["text"])
        except:
            origin = " "
        try:
            spread = " ".join(memes['content'].iloc[i]["spread"]["text"])
        except:
            spread = " "

    meme = {"title":title,"origin":extract_events_spacy(origin),"spread": extract_events_spacy(spread) }
    out.append(meme)

 42%|██████████████████████████▏                                    | 6403/15406 [13:20<18:46,  7.99it/s]


KeyboardInterrupt: 

In [ ]:
'''
out = [

{"title": Meme Title,
"origin": `list` of events in origin text,
"spread": `list` of events in spread text}

...
]


'''

In [ ]:
with open(output_json_file, 'w') as fout:
    json.dump(out, fout)